In [6]:
import pandas as pd
import plotly.express as px

In [4]:
df = pd.read_parquet('shopping_carts.parquet')

In [5]:
df.describe()

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip,aisle_id,department_id
count,6.132669e+06,6.132669e+06,6.132669e+06,6.132669e+06,6.132669e+06,6.132669e+06,6.132669e+06,6.132669e+06,5.757096e+06,6.132669e+06,6.132669e+06,6.132669e+06
mean,3.034141e+05,2.555715e+04,8.347707e+00,5.891352e-01,1.034604e+05,1.708789e+01,2.736021e+00,1.343908e+01,1.138992e+01,4.474964e-01,7.137251e+01,9.941165e+00
std,1.751674e+05,1.409440e+04,7.132200e+00,4.919908e-01,5.942233e+04,1.749323e+01,2.097220e+00,4.256280e+00,8.934196e+00,4.972358e-01,3.817693e+01,6.286584e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,3.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
25%,1.517370e+05,1.351200e+04,3.000000e+00,0.000000e+00,5.243000e+04,5.000000e+00,1.000000e+00,1.000000e+01,5.000000e+00,0.000000e+00,3.100000e+01,4.000000e+00
50%,3.032450e+05,2.523000e+04,6.000000e+00,1.000000e+00,1.032530e+05,1.100000e+01,3.000000e+00,1.300000e+01,8.000000e+00,0.000000e+00,8.300000e+01,9.000000e+00
75%,4.551400e+05,3.786600e+04,1.100000e+01,1.000000e+00,1.548280e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01,1.000000e+00,1.070000e+02,1.600000e+01
max,6.069150e+05,4.968800e+04,1.450000e+02,1.000000e+00,2.062010e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01,1.000000e+00,1.340000e+02,2.100000e+01


In [13]:
# check for incomplete order_ids
df['days_since_prior_order'].isna().sum() # first order has no days_since_prior_order

375573

In [28]:
# check for inconsistence days_since_prior_order
df.groupby('order_id')['days_since_prior_order'].count().max() # 1
# check where therer are 0 unique values when grouped by order_id

df_merged = df.merge(df.groupby('order_id')['days_since_prior_order'].nunique(), on='order_id').rename(columns={"days_since_prior_order_y": "nunique_dspo"})
df_merged[df_merged['nunique_dspo'] != 1]

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order_x,tip,product_name,aisle_id,department_id,department,aisle,county,nunique_dspo
71,13,46383,15,0,106387,1,5,19,NaN,0,Imported Capote Capers,110,13,pantry,pickled goods olives,San Mateo,0
72,13,21019,14,0,106387,1,5,19,NaN,0,Roasted Red Pepper Hummus,67,20,deli,fresh dips tapenades,San Mateo,0
73,13,33768,7,0,106387,1,5,19,NaN,0,Sinfully Sweet Campari Tomatoes,83,4,produce,fresh vegetables,San Mateo,0
74,13,25195,16,0,106387,1,5,19,NaN,0,Roasted Garlic Loaf,112,3,bakery,bread,San Mateo,0
75,13,40709,8,0,106387,1,5,19,NaN,0,Sea Salt & Vinegar Potato Chips,107,19,snacks,chips pretzels,San Mateo,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6132445,606893,38768,2,0,92134,1,5,10,NaN,0,Sweet Kale Salad Mix,32,4,produce,packaged produce,Stanislaus,0
6132665,606915,27966,1,0,107102,1,3,10,NaN,0,Organic Raspberries,123,4,produce,packaged vegetables fruits,Amador,0
6132666,606915,27845,4,0,107102,1,3,10,NaN,0,Organic Whole Milk,84,16,dairy eggs,milk,Amador,0
6132667,606915,26940,3,0,107102,1,3,10,NaN,0,Organic Large Green Asparagus,83,4,produce,fresh vegetables,Amador,0


In [31]:
# check if there is a days_since_prior_order value of NaN when there is a earlier order_id of the same user
df_merged.sort_values(by=['user_id', 'order_id']).groupby('user_id')['days_since_prior_order_x'].shift(1).isna().sum() # 0

408528